# Generating simulated data

Instructions:
- Copy any existing hdf5 file. Do NOT use the original, the content of the file will be overwritten!
- Rename it according to: static\<animal_id>-\<session>-\<scan_idx>-preproc0.h5
    - animal_id should be fixed to 0 for simulated datasets
    - session should indicate datasets with the same way of general noise generation , i.e. independent poisson, noise correlations, brainstate simulations...
    - scan_idx should indicate datasets with different neurons, i.e. different parameter values for the (gabor) filter generation
- Run all cells ("Analyze simulated data" not necessary)

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import h5py
from functools import partial
from mlutils.data.datasets import StaticImageSet

In [ ]:
# Choose the session and scan_idx of your simulated data. Animal_id must always be 0 for simulated data!
animal_id = 0
session = 1
scan_idx= 0

In [ ]:
# Load hdf5 file
path = 'static{}-{}-{}-preproc0.h5'.format(animal_id, session, scan_idx)
f = h5py.File(path, 'r+')

# Add keys to store parameters of simulated data
try:
    f['simulation_info/ground_truths'] = np.full_like(f['responses'][:], None)
    f['simulation_info/theta'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8')    
    f['simulation_info/sigma'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8')
    f['simulation_info/Lambda'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8')
    f['simulation_info/psi'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8') 
    f['simulation_info/gamma'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8')
    f['simulation_info/center'] = np.full((len(f['neurons']['unit_ids'][:]), 2), None).astype('<f8')
    f['simulation_info/size'] = np.full((len(f['neurons']['unit_ids'][:]), 2), None).astype('<f8')
    f['simulation_info/normalize'] = np.full((len(f['neurons']['unit_ids'][:])), None).astype('<f8')
    f['simulation_info/seed'] = np.full((1), None).astype('<f8')
except: 
    print('Error occurred trying to add keys')
    pass

## Define necessary functions

In [ ]:
def gabor_fn(theta, sigma=2, Lambda=10, psi=np.pi/2, gamma=.8, center=(0, 0), size=(36, 64), normalize=True):
    """Returns a gabor filter.

    Args:
        theta (float): Orientation of the sinusoid (in ratian).
        sigma (float): std deviation of the Gaussian.
        Lambda (float): Sinusoid wavelengh (1/frequency).
        psi (float): Phase of the sinusoid.
        gamma (float): The ratio between sigma in x-dim over sigma in y-dim (acts
            like an aspect ratio of the Gaussian).
        center (tuple of integers): The position of the filter.
        size (tuple of integers): Image height and width.
        normalize (bool): Whether to normalize the entries. This is computed by
            dividing each entry by the root sum squared of the whole image.

    Returns:
        2D Numpy array: A gabor filter.

    """

    sigma_x = sigma
    sigma_y = sigma / gamma

    xmax, ymax = size
    xmax, ymax = (xmax - 1)/2, (ymax - 1)/2
    xmin = -xmax
    ymin = -ymax
    (y, x) = np.meshgrid(np.arange(ymin, ymax+1), np.arange(xmin, xmax+1))

    # shift the positon
    y -= center[0]
    x -= center[1]

    # Rotation
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)

    gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi / Lambda * x_theta + psi)

    if normalize:
        # root sum squared
        gb /= np.sqrt(np.sum(gb ** 2))
        # make sure the sum is equal to zero
        # gb[gb > 0] = gb[gb > 0] * (np.abs(gb[gb < 0].sum()) / gb[gb > 0].sum())
        gb -= gb.mean()

    return gb

def _elu(val, alpha=1.):
    return val if val > 0 else alpha * (np.exp(val) - 1)
elu1 = lambda arr, alpha=1.: np.array(list(map(partial(_elu, alpha=alpha), arr))) + 1

def compute_linear_out(img, filters):
    out = img * filters
    return out.sum(axis=(1, 2))

def compute_statistics(data):
    return np.max(data, axis=0), np.mean(data, axis=0), np.median(data, axis=0), np.min(data, axis=0), np.std(data, axis=0)

## Create gabor filters

In [ ]:
seed = 1
np.random.seed(seed)

In [ ]:
n_neurons = len(f['neurons']['unit_ids'][:])

# leave 10 pixes space between the center of the Gabor and the edge of the image
# X dimension: 64
xs = np.random.randint(low=-22, high=23, size=n_neurons)
# Y dimension: 32
ys = np.random.randint(low=-8, high=9, size=n_neurons)
center = np.vstack([xs,ys]).T

# Angles and phases range from 0 to 2 pi
theta = np.random.uniform(low=0, high=2, size=n_neurons) * np.pi
psi = np.random.uniform(low=0, high=2, size=n_neurons) * np.pi

# keep like default 
sigma = np.full((n_neurons), 2)
Lambda = np.full((n_neurons), 10)
gamma = np.full((n_neurons), .8)
size = np.full((n_neurons, 2), np.squeeze(f['images'][0]).shape)
normalize = np.full((n_neurons), True)

filters = []
for n in range(n_neurons):
    Filter = gabor_fn(theta=theta[n], sigma=sigma[n], Lambda=Lambda[n], psi=psi[n], gamma=gamma[n], center=center[n], size=size[n], normalize=normalize[n])
    filters.append(Filter)
filters = np.array(filters)

In [ ]:
# show 3 example filters
for i, gabor in enumerate(filters[:3]):
    plt.imshow(gabor, vmin=-np.abs(gabor.max()), vmax=np.abs(gabor.max()), cmap=plt.cm.bwr)
    plt.show()

### If you want to, create random images based on images statistics

In [ ]:
# This takes long. Pre-fitted distribution in the next cell. 

# beta = stats.beta

# pixels = f['images'][:].flatten()
# y = pixels

# x = np.linspace(0, y.max(), len(y))
# # fit
# param = beta.fit(y)
# pdf_fitted = beta.pdf(x, *param)
# plt.plot(x, pdf_fitted, color='r')

# # plot the histogram
# plt.hist(y, normed=True, bins=len(np.unique(pixels)))

# plt.show()

In [ ]:
# Sample new images from distribution
params = (1.595, 2.203, -2.265, 272.882) # fitted beta distribution params for pixels of mouse static natural images 
fitted_beta = stats.beta(*params)

beta_images = np.zeros(f['images'][:].shape)
for i, image_id in enumerate(f['item_info']['frame_image_id'][:]):
    # Same image_id gets the same random image by setting the random seed to the same value for same image_id
    random_state = np.random.get_state()
    np.random.seed(image_id)
    
    beta_images[i][0] = fitted_beta.rvs(beta_images[i][0].shape)
    
    np.random.set_state(random_state)

# Force into pixel interval and round to integers 
maximum = f['images'][:].max()
minimum = f['images'][:].min()
beta_images[beta_images > maximum] = maximum
beta_images[beta_images < minimum] = minimum
beta_images = np.round(beta_images)

## Simulate neurons

In [ ]:
# Choose the input images for the simulated data
image_source = 'beta distribution'

if image_source == 'old_dataset':
    images = f['images'][:]   # use the images that were in the initial hdf5 file
elif image_source == 'beta distribution':
    images = beta_images        # use the images drawn from the beta distribution
else:
    raise ValueError('Image source not defined')

In [ ]:
scale=0.3 # scale the responses to be in the range of real data. This parameter can be adjusted freely

ground_truths= []
for image in tqdm(images):
    linear_out = compute_linear_out(img=image, filters=filters) * scale
    ground_truth = elu1(linear_out)    
    ground_truths.append(ground_truth)
    
ground_truths = np.array(ground_truths)
responses = np.random.poisson(ground_truths)

## Analize simulated data (not necessary)

#### Responses of one neuron to all images

In [ ]:
dataset = StaticImageSet('/notebooks/data/static20457-5-9-preproc0.h5', "images", "responses")
dat_responses = dataset.responses.copy()
print('Maximum response in simulated data: {}'.format(responses.max()))
print('Maximum response in real data: {}'.format(dat_responses.max()))

# s = np.array(dataset.statistics["responses"]['all']["std"])

# threshold = 0.01 * s.mean()
# idx = s > threshold
# _response_precision = np.ones_like(s) / threshold
# _response_precision[idx] = 1 / s[idx]

# dat_responses = (dat_responses * _response_precision)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 5), dpi=150)

axes[0].plot(responses[:,0])
axes[0].set_title('Simulated Neuron')
axes[0].set_xlabel('Image')

axes[1].plot(dat_responses[:,0])
axes[1].set_title('Real Neuron')
axes[1].set_xlabel('Image')
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 5), dpi=150)

axes[0].plot(responses[dataset.tiers=='test'][:,0])
axes[0].set_title('Simulated Neuron')
axes[0].set_xlabel('Image')

axes[1].plot(dat_responses[dataset.tiers=='test'][:,0])
axes[1].set_title('Real Neuron')
axes[1].set_xlabel('Image')
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 5), dpi=150)

i=1

axes[0].plot(responses[dataset.info.frame_image_id == 2214][:,i])
axes[0].set_title('Simulated Neuron')
axes[0].set_xlabel('Image')

axes[1].plot(dat_responses[dataset.info.frame_image_id == 2214][:,i])
axes[1].set_title('Real Neuron')
axes[1].set_xlabel('Image')
plt.tight_layout()                        

#### Response of all neurons to one image

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 5), dpi=150)

axes[0].plot(responses[0,:])
axes[0].set_title('Simulated Neurons')
axes[0].set_xlabel('Neuron')

axes[1].plot(dat_responses[0,:])
axes[1].set_title('Real Neurons')
axes[1].set_xlabel('Neuron')
plt.tight_layout()

#### Maximum responses of all neurons to all images

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(5, 3), dpi=150)

axes[0].hist(responses.max(axis=0))
axes[0].set_title('Simulated Neurons')
axes[0].set_xlabel('Max response')

axes[1].hist(dat_responses.max(axis=0))
axes[1].set_title('Real Neurons')
axes[1].set_xlabel('Max response')
plt.tight_layout()

## Write to hdf5 file

In [ ]:
# Overwrite information that does not exist in simulated data
f['pupil_center'][:]                         = np.full_like(f['pupil_center'][:], None)
f['behavior'][:]                             = np.full_like(f['behavior'], None)
f['trial_idx'][:]                            = np.full_like(f['trial_idx'], 0)
f['item_info']['trial_idx'][:]               = np.full_like(f['item_info']['trial_idx'][:], 0)
f['item_info']['frame_last_flip'][:]         = np.full_like(f['item_info']['frame_last_flip'][:], 0)
f['item_info']['frame_pre_blank_period'][:]  = np.full_like(f['item_info']['frame_pre_blank_period'][:], 0)
f['item_info']['frame_presentation_time'][:] = np.full_like(f['item_info']['frame_presentation_time'][:], 0)
f['item_info']['frame_trial_ts'][:]          = np.full_like(f['item_info']['frame_trial_ts'][:], 0)

# Overwrite responses and ground truth
f['responses'][:] = responses
if image_source == 'old_dataset':
    pass
elif image_source == 'beta distribution':
    f['images'][:] = images
else:
    raise ValueError('Image source not defined')

#Write information that is only present in simulated data
f['simulation_info']['ground_truths'][:] = ground_truths
f['simulation_info']['theta'][:] = theta  
f['simulation_info']['sigma'][:] = sigma
f['simulation_info']['Lambda'][:] = Lambda
f['simulation_info']['psi'][:] = psi
f['simulation_info']['gamma'][:] = gamma
f['simulation_info']['center'][:] = center
f['simulation_info']['size'][:] = size
f['simulation_info']['normalize'][:] = normalize
#f['simulation_info']['seed'][:] = seed
    
# Compute and overwrite statistics
for key in f['statistics'].keys():
    if not key in ('images',  'responses'):
        del f['statistics'][key] 
        
maximum, mean, median, minimum, std = compute_statistics(responses[f['tiers'][:] == b'train']) 
f['statistics']['responses']['all']['max'][:]    = maximum
f['statistics']['responses']['all']['mean'][:]   = mean
f['statistics']['responses']['all']['median'][:] = median
f['statistics']['responses']['all']['min'][:]    = minimum
f['statistics']['responses']['all']['std'][:]    = std

maximum, mean, median, minimum, std = compute_statistics(responses[(f['types'][:] == b'stimulus.Frame') & (f['tiers'][:] == b'train')]) 
f['statistics']['responses']['stimulus.Frame']['max'][:]    = maximum
f['statistics']['responses']['stimulus.Frame']['mean'][:]   = mean
f['statistics']['responses']['stimulus.Frame']['median'][:] = median
f['statistics']['responses']['stimulus.Frame']['min'][:]    = minimum
f['statistics']['responses']['stimulus.Frame']['std'][:]    = std

# Overwrite dataset information
f['neurons']['animal_ids'][:] = np.full_like(f['neurons']['animal_ids'][:], animal_id)
f['neurons']['scan_idx'][:]   = np.full_like(f['neurons']['scan_idx'][:], scan_idx)
f['neurons']['sessions'][:]   = np.full_like(f['neurons']['sessions'][:], session)

f['neurons']['area'][:]       = np.full_like(f['neurons']['area'][:], b'si') #simulated
f['neurons']['layer'][:]      = np.full_like(f['neurons']['layer'][:], b'simu') #simulated

f['item_info']['animal_id'][:]= np.full_like(f['item_info']['animal_id'][:], animal_id)
f['item_info']['session'][:]  = np.full_like(f['item_info']['session'][:], session)
f['item_info']['scan_idx'][:] = np.full_like(f['item_info']['scan_idx'][:], scan_idx)


In [ ]:
f.close()

### Check oracles

In [ ]:
from mlutils.data.datasets import StaticImageSet
from torch.utils.data import DataLoader
from data.loader_configurators import RepeatsBatchSampler
from mlutils.measures import corr

dataset = StaticImageSet(path, "images", "responses")

types = np.unique(dataset.types)
if len(types) == 1 and types[0] == "stimulus.Frame":
    condition_hashes = dataset.info.frame_image_id
else:
    raise ValueError("Do not recognize types={}".format(*types))

loader = DataLoader(dataset, sampler=RepeatsBatchSampler(condition_hashes))

# --- compute oracles
oracles, data = [], []
for inputs, outputs in loader:
    inputs = np.squeeze(inputs.cpu().numpy(), axis=0)
    outputs = np.squeeze(outputs.cpu().numpy(), axis=0)
    r, n = outputs.shape  # number of frame repeats, number of neurons
    if r < 4:  # minimum number of frame repeats to be considered for oracle, free choice
        continue
    assert np.all(np.abs(np.diff(inputs, axis=0)) == 0), "Images of oracle trials do not match"

    mu = outputs.mean(axis=0, keepdims=True)
    oracle = (mu - outputs / r) * r / (r - 1)
    oracles.append(oracle)
    data.append(outputs)

assert len(data) > 0, "Found no oracle trials!"
pearson = corr(np.vstack(data), np.vstack(oracles), axis=0)
unit_ids = dataset.neurons.unit_ids
assert len(unit_ids) == len(pearson) == outputs.shape[-1], "Neuron numbers do not add up"
print(pearson)